<h1 align=center><font size = 6> Crop Yield Prediction. </font></h1>

## Import required libraries. 

In [99]:
import numpy as np    #Library to handle data in vectorized manner.
import pandas as pd   #library for data analysis.

#Plotting libray matplotlib and associated ploting modules.
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

import pylab as pl

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

from sklearn import preprocessing

from sklearn.ensemble import RandomForestRegressor

from sklearn import svm


%matplotlib inline
print('All the required libraries are imported.....!')

All the required libraries are imported.....!


## Data Prepration.

### Read crop production dataset into pandas dataframe.

In [100]:
df_crop = pd.read_csv("crop_production.csv")

# Now look at the data frame. 
df_crop.head()

,District_name,RICE,WHEAT,JOWARR,BAJRA,MAIZE,RAGI,GRAM,TUR,UDID,MUNG
0,Ahemadnagar,21165,81820,290528,223329,120126,2085,28000,11705,6161,328
1,Akola,0,30569,104344,0,0,0,12632,20821,7377,28576
2,Amravati,4773,15209,0,0,0,0,66270,110996,3134,26431
3,Aurangabad,0,74633,79673,80819,417410,0,28673,28388,667,862
4,Beed,189,50908,119590,103479,6048,0,28066,30039,8738,2538


### Replace blank cells with nan and remove these nan values.

In [101]:
nan_value = float("NaN")
df_crop.replace("", nan_value, inplace=True)
df_crop.dropna(subset = ["District_name"], inplace=True)
print("Drop Sucessfull")
#now look at the shape of our data frame.
print("Size of the data frame is :", df_crop.shape)

Drop Sucessfull
Size of the data frame is : (33, 11)


### Read whether dataset into pandas dataframe.

In [102]:
df_whether = pd.read_csv("whether1.csv")

# Now look at the data frame. 
df_whether.head()

,District_name,Avg_rainfall,Avg_temperature
0,Ahemadnagar,345.0,24.0
1,Akola,602.0,27.0
2,Amravati,808.0,27.0
3,Aurangabad,741.0,26.0
4,Beed,692.0,29.0


### Replace blank cells with nan and remove these nan values.

In [103]:
nan_value = float("NaN")
df_whether.replace("", nan_value, inplace=True)
df_whether.dropna(subset = ["District_name"], inplace=True)
print("Drop Sucessfull")
#now look at the shape of our data frame.
print("Size of the data frame is :", df_whether.shape)

Drop Sucessfull
Size of the data frame is : (33, 3)



### Now lets merge the dataframes.

In [104]:
df_yield = df_crop.merge(df_whether, on="District_name", how="left")

#lets have a look on dataframe.
df_yield.head()

,District_name,RICE,WHEAT,JOWARR,BAJRA,MAIZE,RAGI,GRAM,TUR,UDID,MUNG,Avg_rainfall,Avg_temperature
0,Ahemadnagar,21165,81820,290528,223329,120126,2085,28000,11705,6161,328,345.0,24.0
1,Akola,0,30569,104344,0,0,0,12632,20821,7377,28576,602.0,27.0
2,Amravati,4773,15209,0,0,0,0,66270,110996,3134,26431,808.0,27.0
3,Aurangabad,0,74633,79673,80819,417410,0,28673,28388,667,862,741.0,26.0
4,Beed,189,50908,119590,103479,6048,0,28066,30039,8738,2538,692.0,29.0


### Set the District name as index in the dataframe.

In [105]:
#Set the district name as index - useful for quickly looking up district using .loc method
#df_yield.set_index('District_name' , inplace = True)

#now see how data frame looks.
#df_yield.head()

In [106]:
df_yield.describe()

,RICE,WHEAT,JOWARR,BAJRA,MAIZE,RAGI,GRAM,TUR,UDID,MUNG,Avg_rainfall,Avg_temperature
count,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000
mean,45512.151515,32683.303030,50022.606061,26030.030303,32734.575758,5161.303030,20601.787879,23596.242424,9915.636364,9151.727273,1106.606061,26.181818
std,98471.792650,35055.380119,76834.435640,57602.035923,79556.506548,15490.197018,23837.159493,37352.356887,29365.134573,25063.183520,734.536458,1.758098
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,345.000000,21.000000
25%,0.000000,601.000000,22.000000,0.000000,0.000000,0.000000,202.000000,408.000000,97.000000,20.000000,730.000000,25.000000
50%,2502.000000,26538.000000,6035.000000,128.000000,0.000000,0.000000,12660.000000,7498.000000,2700.000000,1152.000000,832.000000,26.000000
75%,21165.000000,50908.000000,79673.000000,9556.000000,17352.000000,2085.000000,28673.000000,30175.000000,6692.000000,7024.000000,1125.000000,27.000000
max,397800.000000,129210.000000,290528.000000,223329.000000,417410.000000,82000.000000,76307.000000,165903.000000,166699.000000,141142.000000,3884.000000,32.000000


### Prediction models :

#### create the result dataframe to store the prediction results :

In [107]:
df_results = df_yield[['District_name']]
df_results["Predicted_rice"] = " "
df_results["Predicted_wheat"] = " "
df_results["Predicted_jowarr"] = " "
df_results["Predicted_bajra"] = " "
df_results["Predicted_maize"] = " "
df_results["Predicted_ragi"] = " "
df_results["Predicted_gram"] = " "
df_results["Predicted_tur"] = " "
df_results["Predicted_udid"] = " "
df_results["Predicted_mung"] = " "

df_results.head()

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\program files\python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\program files\python36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

,District_name,Predicted_rice,Predicted_wheat,Predicted_jowarr,Predicted_bajra,Predicted_maize,Predicted_ragi,Predicted_gram,Predicted_tur,Predicted_udid,Predicted_mung
0,Ahemadnagar,,,,,,,,,,
1,Akola,,,,,,,,,,
2,Amravati,,,,,,,,,,
3,Aurangabad,,,,,,,,,,
4,Beed,,,,,,,,,,


### Note : we used diffrent algorithms to predict the crop yeilds.

### 1. Rice prediction model : by using RandomForestRegressor()

In [108]:
error=[]
pred = []
index=0
for i in range(0,33):
  
  X = df_yield[['Avg_rainfall', 'Avg_temperature']].drop(i) 
  Y = df_yield['RICE'].drop(i)

  x_train,x_test,y_train,y_test=train_test_split(X,Y, test_size=0.2, random_state=1)
  reg=RandomForestRegressor()
  reg.fit(x_train,y_train)
  

  y_pred=reg.predict([df_yield.loc[i,'Avg_rainfall':'Avg_temperature']])
  print('~ index:',index,'-\n','Predicted Production:',y_pred,', ','Actual Production:',[df_yield.loc[i,'RICE']],', ','Error: ',y_pred-df_yield.loc[i,'RICE'])
  error.append(float(y_pred-df_yield.loc[i,'RICE']))
  
  pred.append(float(y_pred))

  index+=1





~ index: 0 -
 Predicted Production: [13304.36] ,  Actual Production: [21165] ,  Error:  [-7860.64]
~ index: 1 -
 Predicted Production: [7443.04] ,  Actual Production: [0] ,  Error:  [7443.04]
~ index: 2 -
 Predicted Production: [295.9] ,  Actual Production: [4773] ,  Error:  [-4477.1]
~ index: 3 -
 Predicted Production: [3582.22] ,  Actual Production: [0] ,  Error:  [3582.22]
~ index: 4 -
 Predicted Production: [4139.13] ,  Actual Production: [189] ,  Error:  [3950.13]
~ index: 5 -
 Predicted Production: [971.97] ,  Actual Production: [40300] ,  Error:  [-39328.03]
~ index: 6 -
 Predicted Production: [3989.89] ,  Actual Production: [0] ,  Error:  [3989.89]
~ index: 7 -
 Predicted Production: [67288.22] ,  Actual Production: [2502] ,  Error:  [64786.22]
~ index: 8 -
 Predicted Production: [309.6] ,  Actual Production: [8105] ,  Error:  [-7795.4]
~ index: 9 -
 Predicted Production: [21336.97] ,  Actual Production: [40532] ,  Error:  [-19195.03]
~ index: 10 -
 Predicted Production: [37714

#### Rice model evaluation :

In [109]:
import math
print(list(error))

total_error=0
for i in error:
  total_error=total_error+(i*i)
  
mean_squared_error=math.sqrt(total_error)/34

print("\nMean Squared Error: ",mean_squared_error)





[-7860.639999999999, 7443.04, -4477.1, 3582.22, 3950.13, -39328.03, 3989.89, 64786.22, -7795.4, -19195.03, 34599.26, 10505.01, 1470.11, 6375.85, 124806.19, -2338.05, -193642.31, 7418.77, 1899.12, -22436.42, -2944.65, 2560.87, 2064.57, -40647.31, 44792.17999999999, 4775.56, 43792.29, -193619.9, 483.98, -236507.94, 55255.34, 1683.83, 114726.59]

Mean Squared Error:  12360.227302258665


In [110]:
#store the predicted results into the result dataframe.
  
df_results[["Predicted_rice"]] =  pd.DataFrame(pred)
df_results.head()



c:\program files\python36\lib\site-packages\pandas\core\frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,District_name,Predicted_rice,Predicted_wheat,Predicted_jowarr,Predicted_bajra,Predicted_maize,Predicted_ragi,Predicted_gram,Predicted_tur,Predicted_udid,Predicted_mung
0,Ahemadnagar,13304.36,,,,,,,,,
1,Akola,7443.04,,,,,,,,,
2,Amravati,295.90,,,,,,,,,
3,Aurangabad,3582.22,,,,,,,,,
4,Beed,4139.13,,,,,,,,,


In [111]:
df_results.shape


(33, 11)

### 2. Wheat Prediction model :  using Multiple Linear Regression.

In [112]:
error=[]
pred = []
index=0
for i in range(0,33):
  
  X = df_yield[['Avg_rainfall', 'Avg_temperature']].drop(i) 
  Y = df_yield['WHEAT'].drop(i)

  x_train,x_test,y_train,y_test=train_test_split(X,Y, test_size=0.2, random_state=1)
  reg=linear_model.LinearRegression()
  reg.fit(x_train,y_train)
  

  y_pred=reg.predict([df_yield.loc[i,'Avg_rainfall':'Avg_temperature']])
  
  error.append(float(y_pred-df_yield.loc[i,'WHEAT']))
  
  pred.append(float(y_pred))

  index+=1


#### Wheat model evaluation.


In [113]:
import math
print(list(error))

total_error=0
for i in error:
  total_error=total_error+(i*i)
  
mean_squared_error=math.sqrt(total_error)/34

print("\nMean Squared Error: ",mean_squared_error)

[-19593.096037087016, 13152.375112013862, 22377.94196589291, -32878.960056105454, -22642.51446039221, 19733.87487331117, -28727.12940874156, 24095.81708095997, -31966.289471521013, 5792.144092956791, 24081.71372562382, 27355.94974662841, -81422.15458709496, 45641.3355153591, -12625.52238188003, 6362.4896082430205, -100314.12646420926, 26389.2568942816, 37496.10507409822, -27781.631638539664, 24740.941516924562, 40446.20493369512, 49552.92363103508, -48112.69220591878, -27674.24301300349, 2909.448859904951, -12574.010905293893, 30454.433122359187, 6964.427454998455, 2149.050297316251, 36595.932378535625, 24243.057574760867, 6945.1705422425875]

Mean Squared Error:  5851.874762974718


In [114]:
#store the predicted results into the result dataframe.
  
df_results[["Predicted_wheat"]] =  pd.DataFrame(pred)
df_results.head()

c:\program files\python36\lib\site-packages\pandas\core\frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,District_name,Predicted_rice,Predicted_wheat,Predicted_jowarr,Predicted_bajra,Predicted_maize,Predicted_ragi,Predicted_gram,Predicted_tur,Predicted_udid,Predicted_mung
0,Ahemadnagar,13304.36,62226.903963,,,,,,,,
1,Akola,7443.04,43721.375112,,,,,,,,
2,Amravati,295.90,37586.941966,,,,,,,,
3,Aurangabad,3582.22,41754.039944,,,,,,,,
4,Beed,4139.13,28265.485540,,,,,,,,


### 3. Jowarr Prediction model :  Multiple linear regression model.

In [115]:
error=[]
pred = []
index=0
for i in range(0,33):
  
  X = df_yield[['Avg_rainfall', 'Avg_temperature']].drop(i) 
  Y = df_yield['JOWARR'].drop(i)

  x_train,x_test,y_train,y_test=train_test_split(X,Y, test_size=0.2, random_state=1)
  reg=linear_model.LinearRegression()
  reg.fit(x_train,y_train)
  

  y_pred=reg.predict([df_yield.loc[i,'Avg_rainfall':'Avg_temperature']])
  
  error.append(float(y_pred-df_yield.loc[i,'JOWARR']))
  
  pred.append(float(y_pred))

  index+=1


#### Jowarr Model Evaluation 

In [116]:
import math
print(list(error))

total_error=0
for i in error:
  total_error=total_error+(i*i)
  
mean_squared_error=math.sqrt(total_error)/34

print("\nMean Squared Error: ",mean_squared_error)

[-192636.81574442502, -37308.95988817752, 56919.956471154, -7531.482361328322, -83965.02105074743, 50010.92338447878, 19788.993352346588, 24189.616054504295, 61530.134003558254, 1387.5337492982508, 21229.879154941824, 55666.10278921446, -163246.68541788653, 89327.00502377126, 23491.343867763877, -22498.41679594794, 44849.01188852481, 25257.983112616464, 60803.40971084591, 65578.76024396764, -146075.43361749838, 60057.84789005818, 82779.5809792775, -192720.80261151207, -130128.78003736062, 69965.70485013211, 54068.54184319358, 37619.95802761399, -105630.9930531574, -56079.0118782465, 57641.90483671494, 50562.90617759357, -31167.349783755664]

Mean Squared Error:  13865.895310861311


In [117]:
#store the predicted results into the result dataframe.
  
df_results[["Predicted_jowarr"]] =  pd.DataFrame(pred) 
df_results.head()

c:\program files\python36\lib\site-packages\pandas\core\frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,District_name,Predicted_rice,Predicted_wheat,Predicted_jowarr,Predicted_bajra,Predicted_maize,Predicted_ragi,Predicted_gram,Predicted_tur,Predicted_udid,Predicted_mung
0,Ahemadnagar,13304.36,62226.903963,97891.184256,,,,,,,
1,Akola,7443.04,43721.375112,67035.040112,,,,,,,
2,Amravati,295.90,37586.941966,56919.956471,,,,,,,
3,Aurangabad,3582.22,41754.039944,72141.517639,,,,,,,
4,Beed,4139.13,28265.485540,35624.978949,,,,,,,


### 4. Bjra Prediction model : Multiple linear regression model 

In [118]:
error=[]
pred = []
index=0
for i in range(0,33):
  
  X = df_yield[['Avg_rainfall', 'Avg_temperature']].drop(i) 
  Y = df_yield['BAJRA'].drop(i)

  x_train,x_test,y_train,y_test=train_test_split(X,Y, test_size=0.2, random_state=1)
  reg=linear_model.LinearRegression()
  reg.fit(x_train,y_train)
  

  y_pred=reg.predict([df_yield.loc[i,'Avg_rainfall':'Avg_temperature']])
  
  error.append(float(y_pred-df_yield.loc[i,'BAJRA']))
  
  pred.append(float(y_pred))

  index+=1


#### Model evaluation : 

In [119]:
import math
print(list(error))

total_error=0
for i in error:
  total_error=total_error+(i*i)
  
mean_squared_error=math.sqrt(total_error)/34

print("\nMean Squared Error: ",mean_squared_error)

[-192271.49951079494, 33708.03367805667, 23364.795232177974, -51246.74416117233, -93261.98851084526, 32053.160871088083, 35996.69155994759, 6317.9606636775425, -85303.15178638152, -6004.751303734432, 2539.6551092409063, 21244.06946581506, -26761.11525516314, 44453.56862789986, 366.248653739749, 35247.981893875636, 19599.700408300618, 18348.116494334274, 27392.940594169515, -184194.24569814128, 42659.158007446094, 34829.02863237774, 49115.57933589042, -88847.04170010774, -62193.881085405475, 44762.24444700839, 34856.15704983659, 19254.137433583237, 31902.219826370303, -17583.292722186947, 24952.987118821504, 31350.331269624352, 31555.868856815563]

Mean Squared Error:  10363.887985771196


In [120]:
#store the predicted results into the result dataframe.
  
df_results[["Predicted_bajra"]] =  pd.DataFrame(pred)
df_results.head()

c:\program files\python36\lib\site-packages\pandas\core\frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,District_name,Predicted_rice,Predicted_wheat,Predicted_jowarr,Predicted_bajra,Predicted_maize,Predicted_ragi,Predicted_gram,Predicted_tur,Predicted_udid,Predicted_mung
0,Ahemadnagar,13304.36,62226.903963,97891.184256,31057.500489,,,,,,
1,Akola,7443.04,43721.375112,67035.040112,33708.033678,,,,,,
2,Amravati,295.90,37586.941966,56919.956471,23364.795232,,,,,,
3,Aurangabad,3582.22,41754.039944,72141.517639,29572.255839,,,,,,
4,Beed,4139.13,28265.485540,35624.978949,10217.011489,,,,,,


### 5. Maize prediction model :  Multiple linear regression model.

In [121]:
error=[]
pred = []
index=0
for i in range(0,33):
  
  X = df_yield[['Avg_rainfall', 'Avg_temperature']].drop(i) 
  Y = df_yield['MAIZE'].drop(i)

  x_train,x_test,y_train,y_test=train_test_split(X,Y, test_size=0.2, random_state=1)
  reg=linear_model.LinearRegression()
  reg.fit(x_train,y_train)
  

  y_pred=reg.predict([df_yield.loc[i,'Avg_rainfall':'Avg_temperature']])
  
  error.append(float(y_pred-df_yield.loc[i,'MAIZE']))
  
  pred.append(float(y_pred))

  index+=1


#### Model evaluation :

In [122]:
import math
print(list(error))

total_error=0
for i in error:
  total_error=total_error+(i*i)
  
mean_squared_error=math.sqrt(total_error)/34

print("\nMean Squared Error: ",mean_squared_error)

[-49480.508791515866, 51309.62511047421, 39387.334698251245, -390146.0612779396, 4268.72113472948, -8293.652548185404, -81496.29654268164, 6419.421405619214, -110634.2588908152, -1178.520744235051, 8007.224233386805, 20001.783452596457, 20619.151756486826, 90888.45894357847, -11715.032486245473, 33653.47697654838, 18422.085625951033, 13903.741494090413, 19232.155452551408, -71797.85880193076, 23006.046164936328, 26276.36774866254, 37633.12357562984, -54856.51349322416, -39735.794542055344, -1263.493311175116, -7517.269304650428, 13606.67774136606, -51401.54076198794, -9586.02544478912, 21075.011020185077, 26000.11024584729, 26799.516653681407]

Mean Squared Error:  13361.315848574335


In [123]:
#store the predicted results into the result dataframe.
  
df_results[["Predicted_maize"]] =  pd.DataFrame(pred)
df_results.head()

c:\program files\python36\lib\site-packages\pandas\core\frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,District_name,Predicted_rice,Predicted_wheat,Predicted_jowarr,Predicted_bajra,Predicted_maize,Predicted_ragi,Predicted_gram,Predicted_tur,Predicted_udid,Predicted_mung
0,Ahemadnagar,13304.36,62226.903963,97891.184256,31057.500489,70645.491208,,,,,
1,Akola,7443.04,43721.375112,67035.040112,33708.033678,51309.625110,,,,,
2,Amravati,295.90,37586.941966,56919.956471,23364.795232,39387.334698,,,,,
3,Aurangabad,3582.22,41754.039944,72141.517639,29572.255839,27263.938722,,,,,
4,Beed,4139.13,28265.485540,35624.978949,10217.011489,10316.721135,,,,,


### 6. Ragi Prediction model :  Multiple linear regression.

In [124]:
error=[]
pred = []
index=0
for i in range(0,33):
  
  X = df_yield[['Avg_rainfall', 'Avg_temperature']].drop(i) 
  Y = df_yield['RAGI'].drop(i)

  x_train,x_test,y_train,y_test=train_test_split(X,Y, test_size=0.2, random_state=1)
  reg=linear_model.LinearRegression()
  reg.fit(x_train,y_train)
  

  y_pred=reg.predict([df_yield.loc[i,'Avg_rainfall':'Avg_temperature']])
  
  error.append(float(y_pred-df_yield.loc[i,'RAGI']))
  
  pred.append(float(y_pred))

  index+=1


#### Model Evaluation :

In [125]:
import math
print(list(error))

total_error=0
for i in error:
  total_error=total_error+(i*i)
  
mean_squared_error=math.sqrt(total_error)/34

print("\nMean Squared Error: ",mean_squared_error)

[-2657.4537872673473, -171.2126338665612, 491.37273346728307, 561.0545174201334, -398.9792842442939, -2853.969659208833, 1178.3309517959106, 1944.4940879590235, 258.4244717113488, 2833.132848122804, 2810.096440993484, 1437.026201829391, 397.9543933437408, 2821.532968203065, -2870.4196693305466, 1082.8581916904513, 1602.911874851718, 1140.4406420771697, 543.8921510907803, -30921.270783191798, 2752.2869638290613, 2454.583920675399, 2673.433797312726, -68238.25711149449, -13532.867506030483, -4643.6978126031645, -1576.2750577346887, 288.4845229543953, 273.3539849121844, 8412.732847188629, 2446.620299585331, 868.7271167870531, 3709.153120326533]

Mean Squared Error:  2276.8625601893623


In [126]:
#store the predicted results into the result dataframe.
  
df_results[["Predicted_ragi"]] =  pd.DataFrame(pred)
df_results.head()

c:\program files\python36\lib\site-packages\pandas\core\frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,District_name,Predicted_rice,Predicted_wheat,Predicted_jowarr,Predicted_bajra,Predicted_maize,Predicted_ragi,Predicted_gram,Predicted_tur,Predicted_udid,Predicted_mung
0,Ahemadnagar,13304.36,62226.903963,97891.184256,31057.500489,70645.491208,-572.453787,,,,
1,Akola,7443.04,43721.375112,67035.040112,33708.033678,51309.625110,-171.212634,,,,
2,Amravati,295.90,37586.941966,56919.956471,23364.795232,39387.334698,491.372733,,,,
3,Aurangabad,3582.22,41754.039944,72141.517639,29572.255839,27263.938722,561.054517,,,,
4,Beed,4139.13,28265.485540,35624.978949,10217.011489,10316.721135,-398.979284,,,,


### 7. Gram Prediction model :  Support vector regressor.

In [127]:
error=[]
pred = []
index=0
for i in range(0,33):
  
  X = df_yield[['Avg_rainfall', 'Avg_temperature']].drop(i) 
  Y = df_yield['GRAM'].drop(i)

  x_train,x_test,y_train,y_test=train_test_split(X,Y, test_size=0.2, random_state=1)
  reg=svm.SVR()
  reg.fit(x_train,y_train)
  

  y_pred=reg.predict([df_yield.loc[i,'Avg_rainfall':'Avg_temperature']])
  
  error.append(float(y_pred-df_yield.loc[i,'GRAM']))
  
  pred.append(float(y_pred))

  index+=1


#### Model evaluation :

In [128]:
import math
print(list(error))

total_error=0
for i in error:
  total_error=total_error+(i*i)
  
mean_squared_error=math.sqrt(total_error)/34

print("\nMean Squared Error: ",mean_squared_error)

[-11511.965291446471, 4470.360023876383, -49782.29528123894, -12185.04955677691, -11577.905481553775, 11302.660951676091, -26956.41358131853, 16793.455253877346, -9822.860456643171, -54176.03664432747, 17072.203462097077, 12935.385625526684, 17101.568345185322, 613.8725218884538, 4439.481967185682, -35129.278168183155, -59820.71119503585, 11644.440174216616, 16948.326413352115, 17099.8594660653, -23259.016812663984, 16769.922838188257, 16801.645800944956, 17099.812650299606, 17099.101388825853, -3611.843792742602, -613.5641551446206, 17099.443082254453, -2416.7751415775383, 19515.352383082245, 19315.014790710524, -50595.927097445674, -20728.894409819524]

Mean Squared Error:  4067.437454393988


In [129]:
#store the predicted results into the result dataframe.
  
df_results[["Predicted_gram"]] =  pd.DataFrame(pred)
df_results.head()

c:\program files\python36\lib\site-packages\pandas\core\frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,District_name,Predicted_rice,Predicted_wheat,Predicted_jowarr,Predicted_bajra,Predicted_maize,Predicted_ragi,Predicted_gram,Predicted_tur,Predicted_udid,Predicted_mung
0,Ahemadnagar,13304.36,62226.903963,97891.184256,31057.500489,70645.491208,-572.453787,16488.034709,,,
1,Akola,7443.04,43721.375112,67035.040112,33708.033678,51309.625110,-171.212634,17102.360024,,,
2,Amravati,295.90,37586.941966,56919.956471,23364.795232,39387.334698,491.372733,16487.704719,,,
3,Aurangabad,3582.22,41754.039944,72141.517639,29572.255839,27263.938722,561.054517,16487.950443,,,
4,Beed,4139.13,28265.485540,35624.978949,10217.011489,10316.721135,-398.979284,16488.094518,,,


### 8. Tur prediction model : Multiple linear regression.

In [130]:
error=[]
pred = []
index=0
for i in range(0,33):
  
  X = df_yield[['Avg_rainfall', 'Avg_temperature']].drop(i) 
  Y = df_yield['TUR'].drop(i)

  x_train,x_test,y_train,y_test=train_test_split(X,Y, test_size=0.2, random_state=1)
  reg=linear_model.LinearRegression()
  reg.fit(x_train,y_train)
  

  y_pred=reg.predict([df_yield.loc[i,'Avg_rainfall':'Avg_temperature']])
  
  error.append(float(y_pred-df_yield.loc[i,'TUR']))
  
  pred.append(float(y_pred))

  index+=1


#### Model Evaluation.

In [131]:
import math
print(list(error))

total_error=0
for i in error:
  total_error=total_error+(i*i)
  
mean_squared_error=math.sqrt(total_error)/34

print("\nMean Squared Error: ",mean_squared_error)

[40442.38847030133, 12207.923860606694, -87176.7816796536, 4541.157944333114, -5202.588882440934, 18619.053293891127, -6112.334816924224, 22591.39709888927, 19957.598002739905, -31597.82161354847, 23230.729670999237, 25586.186543356816, 22502.761175616746, 26799.38314535038, 22611.084440195176, -139543.33986720166, -22382.84145716847, 22636.46415000086, 30120.163338191094, 25109.607452729586, -40789.30059124, 30745.711707296592, 36089.91428052046, -23232.997515597293, -9987.174617104785, 39716.67343620288, 34155.00820226452, 25227.506582957212, 29380.815730657407, 7992.2053903697815, 27926.02924298413, -768.5356270205084, -57152.05994744653]

Mean Squared Error:  6549.138315457139


In [132]:
#store the predicted results into the result dataframe.
  
df_results[["Predicted_tur"]] =  pd.DataFrame(pred)
df_results.head()

c:\program files\python36\lib\site-packages\pandas\core\frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,District_name,Predicted_rice,Predicted_wheat,Predicted_jowarr,Predicted_bajra,Predicted_maize,Predicted_ragi,Predicted_gram,Predicted_tur,Predicted_udid,Predicted_mung
0,Ahemadnagar,13304.36,62226.903963,97891.184256,31057.500489,70645.491208,-572.453787,16488.034709,52147.388470,,
1,Akola,7443.04,43721.375112,67035.040112,33708.033678,51309.625110,-171.212634,17102.360024,33028.923861,,
2,Amravati,295.90,37586.941966,56919.956471,23364.795232,39387.334698,491.372733,16487.704719,23819.218320,,
3,Aurangabad,3582.22,41754.039944,72141.517639,29572.255839,27263.938722,561.054517,16487.950443,32929.157944,,
4,Beed,4139.13,28265.485540,35624.978949,10217.011489,10316.721135,-398.979284,16488.094518,24836.411118,,


### 9. Udid prediction model : Support vector regressor.

In [133]:
error=[]
pred = []
index=0
for i in range(0,33):
  
  X = df_yield[['Avg_rainfall', 'Avg_temperature']].drop(i) 
  Y = df_yield['UDID'].drop(i)

  x_train,x_test,y_train,y_test=train_test_split(X,Y, test_size=0.2, random_state=1)
  reg=svm.SVR()
  reg.fit(x_train,y_train)
  

  y_pred=reg.predict([df_yield.loc[i,'Avg_rainfall':'Avg_temperature']])
  
  error.append(float(y_pred-df_yield.loc[i,'UDID']))
  
  pred.append(float(y_pred))

  index+=1


#### Model evaluation.

In [134]:
import math
print(list(error))

total_error=0
for i in error:
  total_error=total_error+(i*i)
  
mean_squared_error=math.sqrt(total_error)/34

print("\nMean Squared Error: ",mean_squared_error)

[-3457.449745580464, -4673.332523756871, -431.3573691227198, 2085.7785833264675, -5984.834358953691, 3000.5719395305105, -38680.989246252204, 2991.277875008251, -9360.72151439484, 2103.1417114263927, 2993.338266235163, 246.48426336973534, -6107.549630290745, -1198.061766417888, 2314.353279777378, -3939.6937050719125, 2753.9378947021237, 813.599520925989, 2988.7180075746824, -13915.78128998173, -19935.57075072882, 2843.058813501339, 3126.3370258695822, 3131.373768605873, 3130.612529702209, 134.9445777052124, 432.02383658598956, 3034.067456385584, 331.47904505706765, 1030.6007722814775, 3130.7658818568966, -163699.73442948726, -788.8411972143776]

Mean Squared Error:  5027.031438545247


In [135]:
#store the predicted results into the result dataframe.
  
df_results[["Predicted_udid"]] =  pd.DataFrame(pred)
df_results.head()

c:\program files\python36\lib\site-packages\pandas\core\frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,District_name,Predicted_rice,Predicted_wheat,Predicted_jowarr,Predicted_bajra,Predicted_maize,Predicted_ragi,Predicted_gram,Predicted_tur,Predicted_udid,Predicted_mung
0,Ahemadnagar,13304.36,62226.903963,97891.184256,31057.500489,70645.491208,-572.453787,16488.034709,52147.388470,2703.550254,
1,Akola,7443.04,43721.375112,67035.040112,33708.033678,51309.625110,-171.212634,17102.360024,33028.923861,2703.667476,
2,Amravati,295.90,37586.941966,56919.956471,23364.795232,39387.334698,491.372733,16487.704719,23819.218320,2702.642631,
3,Aurangabad,3582.22,41754.039944,72141.517639,29572.255839,27263.938722,561.054517,16487.950443,32929.157944,2752.778583,
4,Beed,4139.13,28265.485540,35624.978949,10217.011489,10316.721135,-398.979284,16488.094518,24836.411118,2753.165641,


### 10. Mung Prediction Model :  by using Support Vector Regressor.

In [136]:
error=[]
pred = []
index=0
for i in range(0,33):
  
  X = df_yield[['Avg_rainfall', 'Avg_temperature']].drop(i) 
  Y = df_yield['MUNG'].drop(i)

  x_train,x_test,y_train,y_test=train_test_split(X,Y, test_size=0.2, random_state=1)
  reg=svm.SVR()
  reg.fit(x_train,y_train)
  

  y_pred=reg.predict([df_yield.loc[i,'Avg_rainfall':'Avg_temperature']])
  
  error.append(float(y_pred-df_yield.loc[i,'MUNG']))
  
  pred.append(float(y_pred))

  index+=1


#### Model evaluation.

In [137]:
import math
print(list(error))

total_error=0
for i in error:
  total_error=total_error+(i*i)
  
mean_squared_error=math.sqrt(total_error)/34

print("\nMean Squared Error: ",mean_squared_error)

[1775.550254419536, -27209.474438829926, -25064.819865893624, 1240.7036333558945, -435.0831069563392, 2312.532029852484, -23400.726707447528, 2301.785100190661, -139039.03630803447, 1682.5232336660388, 2304.762822141366, -208.17523260326152, -6757.456064575181, -9219.188786109244, 1154.8640374613128, -2824.5420413343204, 1732.467630393427, 944.1109023300664, 2169.887479315441, -4716.163770489522, -5836.989269367286, 2349.433052766659, 2480.4960169356864, 2496.0919660437426, 2495.3313182329825, -189.2450177306555, 397.71068836200675, 2476.8019518873866, 1973.8168155809217, 2495.559828295839, 2490.5422365233753, -17023.903199251574, -3209.414957537625]

Mean Squared Error:  4344.939241485701


In [138]:
#store the predicted results into the result dataframe.
  
df_results[["Predicted_mung"]] =  pd.DataFrame(pred)
df_results.head()

c:\program files\python36\lib\site-packages\pandas\core\frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,District_name,Predicted_rice,Predicted_wheat,Predicted_jowarr,Predicted_bajra,Predicted_maize,Predicted_ragi,Predicted_gram,Predicted_tur,Predicted_udid,Predicted_mung
0,Ahemadnagar,13304.36,62226.903963,97891.184256,31057.500489,70645.491208,-572.453787,16488.034709,52147.388470,2703.550254,2103.550254
1,Akola,7443.04,43721.375112,67035.040112,33708.033678,51309.625110,-171.212634,17102.360024,33028.923861,2703.667476,1366.525561
2,Amravati,295.90,37586.941966,56919.956471,23364.795232,39387.334698,491.372733,16487.704719,23819.218320,2702.642631,1366.180134
3,Aurangabad,3582.22,41754.039944,72141.517639,29572.255839,27263.938722,561.054517,16487.950443,32929.157944,2752.778583,2102.703633
4,Beed,4139.13,28265.485540,35624.978949,10217.011489,10316.721135,-398.979284,16488.094518,24836.411118,2753.165641,2102.916893


### Set district name as index name :

In [139]:
#Set the district name as index - useful for quickly looking up district using .loc method
df_results.set_index('District_name' , inplace = True)

#now see how data frame looks.
df_results.head()

,Predicted_rice,Predicted_wheat,Predicted_jowarr,Predicted_bajra,Predicted_maize,Predicted_ragi,Predicted_gram,Predicted_tur,Predicted_udid,Predicted_mung
District_name,,,,,,,,,,
Ahemadnagar,13304.36,62226.903963,97891.184256,31057.500489,70645.491208,-572.453787,16488.034709,52147.388470,2703.550254,2103.550254
Akola,7443.04,43721.375112,67035.040112,33708.033678,51309.625110,-171.212634,17102.360024,33028.923861,2703.667476,1366.525561
Amravati,295.90,37586.941966,56919.956471,23364.795232,39387.334698,491.372733,16487.704719,23819.218320,2702.642631,1366.180134
Aurangabad,3582.22,41754.039944,72141.517639,29572.255839,27263.938722,561.054517,16487.950443,32929.157944,2752.778583,2102.703633
Beed,4139.13,28265.485540,35624.978949,10217.011489,10316.721135,-398.979284,16488.094518,24836.411118,2753.165641,2102.916893


### Now replace all negative values with 0 in a dataframe.

In [140]:
df_results[df_results < 0] = 0
df_results.head()

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\program files\python36\lib\site-packages\pandas\core\frame.py:3530: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


,Predicted_rice,Predicted_wheat,Predicted_jowarr,Predicted_bajra,Predicted_maize,Predicted_ragi,Predicted_gram,Predicted_tur,Predicted_udid,Predicted_mung
District_name,,,,,,,,,,
Ahemadnagar,13304.36,62226.903963,97891.184256,31057.500489,70645.491208,0.000000,16488.034709,52147.388470,2703.550254,2103.550254
Akola,7443.04,43721.375112,67035.040112,33708.033678,51309.625110,0.000000,17102.360024,33028.923861,2703.667476,1366.525561
Amravati,295.90,37586.941966,56919.956471,23364.795232,39387.334698,491.372733,16487.704719,23819.218320,2702.642631,1366.180134
Aurangabad,3582.22,41754.039944,72141.517639,29572.255839,27263.938722,561.054517,16487.950443,32929.157944,2752.778583,2102.703633
Beed,4139.13,28265.485540,35624.978949,10217.011489,10316.721135,0.000000,16488.094518,24836.411118,2753.165641,2102.916893


### Convert all the values to int :

In [141]:
cols = ['Predicted_rice' ,'Predicted_wheat' ,'Predicted_jowarr' ,'Predicted_bajra' ,'Predicted_maize' ,'Predicted_ragi' ,'Predicted_gram' , 'Predicted_tur' ,'Predicted_udid' ,'Predicted_mung']
df_results[cols] = df_results[cols].applymap(np.int64)

df_results.head()

,Predicted_rice,Predicted_wheat,Predicted_jowarr,Predicted_bajra,Predicted_maize,Predicted_ragi,Predicted_gram,Predicted_tur,Predicted_udid,Predicted_mung
District_name,,,,,,,,,,
Ahemadnagar,13304,62226,97891,31057,70645,0,16488,52147,2703,2103
Akola,7443,43721,67035,33708,51309,0,17102,33028,2703,1366
Amravati,295,37586,56919,23364,39387,491,16487,23819,2702,1366
Aurangabad,3582,41754,72141,29572,27263,561,16487,32929,2752,2102
Beed,4139,28265,35624,10217,10316,0,16488,24836,2753,2102


### Store the result data into CSV file :

In [142]:
df_results.to_csv('Result.csv')
print("Data Successfully Stored....! ")

Data Successfully Stored....! 
